In [ ]:
token = "" #@param {type:"string"}

In [ ]:
print("Installing RelevanceAI ...")
!pip install -q RelevanceAI

from relevanceai import Client

from relevanceai.utils import decode_workflow_token
config = decode_workflow_token(token)
    
authorizationToken = config['authorizationToken']
dataset_id = config["dataset_id"]
start_date_field = config['start_date_field']
end_date_field = config['end_date_field']
day_count_field= config['day_count_field']

In [ ]:
client = Client(token=authorizationToken)
ds = client.Dataset(dataset_id)

In [ ]:
from datetime import datetime
from typing import List, Dict, Any
from relevanceai.utils.doc_utils import DocUtils

document = Dict[str, Any]
documents = List[document]

class CPUWorkflow(DocUtils):
    def __init__(self, start_date_field: str, end_date_field:str, day_count_field:str):
        self.start_date_field = start_date_field
        self.end_date_field = end_date_field
        self.day_count_field = day_count_field

    def run(self, documents: documents) -> documents:
        docs = self.get_fields_across_documents(
            fields=[self.start_date_field, self.end_date_field],
            docs=documents,
        )
        values =[]
        for doc in docs:
            try:
                diff = (datetime.strptime(doc[1], "%Y-%m-%d") - datetime.strptime(doc[0], "%Y-%m-%d")).days
            except:
                diff = None
            values.append(diff)
        self.set_field_across_documents(
            field=self.day_count_field,
            values=values,
            docs=documents,
        )
        return documents

cpu_workflow = CPUWorkflow(
    start_date_field= start_date_field, 
    end_date_field= end_date_field,
    day_count_field= day_count_field)

ds.bulk_apply(    
    bulk_func=cpu_workflow.run,
    update_workers=8,  
    push_workers=8,
    multithreaded_update=True, 
)